# Build a Dashboard Application with Plotly Dash
In this lab, you will be building a Plotly Dash application for users to perform interactive visual analytics on SpaceX launch data in
real-time.

This dashboard application contains input components such as a dropdown list and a range slider to
interact with a pie chart and a scatter point chart. You will be guided to build this dashboard application via the following tasks:

TASK 1: Add a Launch Site Drop-down Input Component
TASK 2: Add a callback function to render success-pie-chart based on selected site dropdown
TASK 3: Add a Range Slider to Select Payload
TASK 4: Add a callback function to render the success-payload-scatter-chart scatter plot
Note:Please take screenshots of the Dashboard and save them. Further upload your notebook to github.

The github url and the screenshots are later required in the presentation slides.

In [25]:

pip install pandas dash

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [29]:
spacex_launch_dash = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

In [30]:
import requests

url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py"
r = requests.get(url)

In [31]:
import pandas as pd

# URL to the SpaceX launch dataset
spacex_launch_dash_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

# Load the dataset
spacex_df = pd.read_csv(spacex_launch_dash_url)

# TASK 1: Add a Launch Site Drop-down Input Component

In [42]:
from dash import Dash, dcc, html
from dash.dependencies import Input, Output

# Assuming spacex_df is already defined and loaded with data
# Initialize the Dash app
app = Dash(__name__)

app.layout = html.Div([
    # Add an H1 HTML component for the Dashboard title
    html.H1("SpaceX Launch Records Dashboard", style={'textAlign': 'center'}),
    
    dcc.Dropdown(
        id='site-dropdown',
        options=[{'label': 'All Sites', 'value': 'ALL'}] +
                [{'label': site, 'value': site} for site in spacex_df['Launch Site'].unique()],
        value='ALL',
        placeholder="Select a Launch Site here",
        searchable=True
    ),
    # Placeholder for additional components like a graph
])

if __name__ == '__main__':
    app.run_server(debug=True)

# TASK 2: Add a callback function to render success-pie-chart based on selected site dropdown

In [43]:
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

# Load your DataFrame here (assuming it's already done above)
# spacex_df = pd.read_csv('path_to_your_spacex_launch_data.csv')

app = Dash(__name__)

app.layout = html.Div([
    html.H1("SpaceX Launch Records Dashboard", style={'textAlign': 'center'}),
    dcc.Dropdown(
        id='site-dropdown',
        options=[
            {'label': 'All Sites', 'value': 'ALL'}
        ] + [{'label': site, 'value': site} for site in spacex_df['Launch Site'].unique()],
        value='ALL',
        placeholder="Select a Launch Site here",
        searchable=True
    ),
    dcc.Graph(id='success-pie-chart'),
])

@app.callback(
    Output('success-pie-chart', 'figure'),
    [Input('site-dropdown', 'value')]
)
def update_success_pie_chart(selected_site):
    if selected_site == 'ALL':
        # Use all rows in the dataframe to calculate success and failure
        fig = px.pie(spacex_df, names='class', title='Total Launch Successes for All Sites')
    else:
        # Filter the dataframe for the selected site
        filtered_df = spacex_df[spacex_df['Launch Site'] == selected_site]
        fig = px.pie(filtered_df, names='class', title=f'Launch Successes for {selected_site}')
    
    # Update pie chart layout for better appearance
    fig.update_traces(textinfo='percent+label')
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)

# TASK 3: Add a Range Slider to Select Payload

In [41]:
from dash import Dash, dcc, html
from dash.dependencies import Input, Output

# Initialize the Dash app (assuming it's already done in the context of your application)
app = Dash(__name__)

# Assuming min_payload and max_payload are determined from the dataset
# For the sake of this example, let's define them statically as per the task
min_payload = 0  # Kg
max_payload = 10000  # Kg

app.layout = html.Div([
    html.H1("SpaceX Launch Records Dashboard", style={'textAlign': 'center'}),
    
    # The launch site dropdown (assuming it's already defined in your layout)
    
    # RangeSlider for selecting payload range
    html.Label("Select Payload Range (Kg):"),
    dcc.RangeSlider(
        id='payload-slider',
        min=0,  # Starting point of the slider
        max=10000,  # Ending point of the slider
        step=1000,  # Interval of the slider
        marks={i: '{}'.format(i) for i in range(0, 10001, 1000)},  # Marks on the slider
        value=[min_payload, max_payload]  # Current selected range
    ),
    
    # Placeholder for additional components like a graph
])

# Callbacks to update the dashboard based on the payload range slider (to be implemented)

if __name__ == '__main__':
    app.run_server(debug=True)

# TASK 4: Add a callback function to render the scatter plot

In [ ]:
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

# Assuming spacex_df is your DataFrame loaded with SpaceX launch data
# Example: spacex_df = pd.read_csv('path_to_your_spacex_launch_data.csv')

app = Dash(__name__)

# Define your app layout including the dropdown for launch sites, range slider for payload, and placeholder for the scatter plot
app.layout = html.Div([
    # Other components like the launch site dropdown and payload range slider here
    
    dcc.Graph(id='success-payload-scatter-chart'),
])

@app.callback(
    Output('success-payload-scatter-chart', 'figure'),
    [Input('site-dropdown', 'value'), Input('payload-slider', 'value')]
)
def update_scatter_plot(selected_site, payload_range):
    filtered_df = spacex_df[
        (spacex_df['Payload Mass (kg)'] >= payload_range[0]) & 
        (spacex_df['Payload Mass (kg)'] <= payload_range[1])
    ]
    
    if selected_site != 'ALL':
        filtered_df = filtered_df[filtered_df['Launch Site'] == selected_site]
    
    fig = px.scatter(
        filtered_df, 
        x='Payload Mass (kg)', 
        y='class', 
        color='Booster Version Category',
        title=f"Launch Outcome by Payload Mass for {selected_site} Site" if selected_site != 'ALL' else "Launch Outcome by Payload Mass for All Sites",
        labels={"class": "Launch Outcome"}
    )
    
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)